## This code plot the three function we are using and compare their behaviours at low and high pT

In [2]:
import ROOT as r
import numpy as np
import math 
import matplotlib.pyplot as plt

In [3]:
pp_file = r.TFile("ROOTfile/pp_pp_5tev.root")
pp_file

In [4]:
Histo = pp_file.Get("Table 6")

In [6]:
P = Histo.Get("Hist1D_y1;1")
TGraphAsymm = Histo.Get("Graph1D_y1;1")

In [8]:
##The goal here is to implement every function and compare it on this interval but Drawn /pt 

In [9]:
## Exponential function

fit_min = 0
fit_max = 5

m_p = 0.938272

def exp_fct(x, par):
    
    pt = x[0]
    A = par[0]
    T = par[1]
    mp = par[2]

    mt = math.sqrt(pt*pt+mp*mp)

    Factor = A*(1/(T*(mp + T))) 

    Expression = math.exp(-(mt-mp)/T)

    return Factor * Expression



In [10]:
exp = r.TF1("exp1", exp_fct, fit_min, fit_max, 3)

In [11]:
#exp parameters

exp.SetParNames("A", "T", "mp")

exp.SetParameters(10, 0.35, 0.938272)

exp.FixParameter(2, 0.938272)

In [12]:
## Levy-Tsallis function

mp = 0.938272

def levy_fct(x, par):
    
    pt = x[0]
    n = par[2]
    T = par[1]
    dNdy = par[0]
    #Beta = par[3]
    mt = math.sqrt( pt*pt + mp*mp ) 

    if (T <= 0) or (n <= 0): #e
        return 0.0
    
    Factor = (n-1)*(n-2)/(n*T*(n*T + mp*(n-2)))

    Arg = 1+((mt-mp)/(n*T))

    return dNdy * (pt) * (n-1)*(n-2)/(n*T*(n*T + mp*(n-2))) * ((1+((mt-mp)/(n*T)))**(-n)) / (pt)


In [13]:
levy = r.TF1("levy", levy_fct, fit_min, fit_max, 3)

In [14]:
#levy parameter

levy.SetParNames("dNdy", "T", "n")

levy.SetParameters(10, 0.35, 6)

In [15]:
## Power-law function

def power_law(x, par):

    pt = x[0]

    C = par[0]
    n = par[1]
    p0 = par[2]

    function = 1 + (pt/p0)
    Factor = function**(-n)

    return C * Factor

In [16]:
power_l = r.TF1("power", power_law, fit_min, fit_max, 3)

In [17]:
#power law parameter

power_l.SetParNames("C", "n", "p0")

power_l.SetParameters(10, 6, 2)


In [ ]:
## I normalize every function and have set their parametersto be coherent.

def normalize (tf1, min, max):

    tf1.SetParameter(0, 1.0)
    integral = tf1.Integral(min, max)
    tf1.SetParameter(0, 1.0/integral)

    return

In [19]:
normalize(exp, fit_min, fit_max)
normalize(levy, fit_min, fit_max)
normalize(power_l, fit_min, fit_max)

In [21]:
## Final plot we the three function

canva_bis = r.TCanvas("canvabis", "comparaison")
canva_bis.SetGrid()
canva_bis.SetLogy()

exp.SetTitle("Comparaison between different laws")
exp.GetXaxis().SetTitle("pt [Gev/c]")
exp.GetYaxis().SetTitle("(1/Nev)*D²(N)/DptDyrap [(Gev/c)^⁻1]")
exp.GetXaxis().SetRangeUser(0., 3.)
exp.SetMinimum(1e-3)

exp.SetLineColor("kRed")
levy.SetLineColor("kBlue")
power_l.SetLineColor("kGreen+2")

exp.SetLineStyle(1)   
levy.SetLineStyle(2)      
power_l.SetLineStyle(3)       

exp.Draw("")
levy.Draw("same")
power_l.Draw("same")


legend = r.TLegend(0.60,0.68,0.88,0.88)

legend.SetTextFont(42)       # police latex propre
legend.SetTextSize(0.035)    # taille lisible
legend.SetMargin(0.25)       # espace symbole ↔ texte

legend.AddEntry(power_l, "Power-law", "l")
legend.AddEntry(levy, "Levy-Tsallis","l")
legend.AddEntry(exp, "Exponential", "l")


legend.Draw()


canva_bis.Draw()

### We can observe that the Levy-Tsallis is a mix between a power-law and an exponential

At low pt --> exponential and levy are adapted

At high pt --> power-law and levy can fit 